In [1]:
#default_exp rescore.feature_extractor

In [2]:
#export
import pandas as pd
import numpy as np

from alphadeep.utils import logging
from alphadeep.pretrained_models import ModelManager
from alphadeep.model.ms2 import calc_ms2_similarity
from alphadeep.mass_spec.match import PepSpecMatch
from alphabase.peptide.fragment import get_charged_frag_types
from alphabase.peptide.precursor import (
    refine_precursor_df
)
from alphabase.peptide.fragment import (
    concat_precursor_fragment_dataframes
)

from alphadeep.rescore.fdr import calc_fdr_for_df

def match_one_raw(
    psm_df_one_raw,
    ms2_file,
    ms2_file_type,
    frag_types_to_match,
    ms2_ppm, ms2_tol,
):
    match = PepSpecMatch(
        charged_frag_types=frag_types_to_match
    )

    # (
    #     psm_df, fragment_mz_df, 
    #     matched_intensity_df, matched_mz_err_df
    # ) = 
    return match.match_ms2_one_raw(
        refine_precursor_df(psm_df_one_raw),
        ms2_file=ms2_file,
        ms2_file_type=ms2_file_type, 
        ppm=ms2_ppm, tol=ms2_tol,
    )

def get_ms2_features(
    psm_df, frag_types,
    predict_intensity_df,
    matched_intensity_df,
    matched_mass_err_df,
)->pd.DataFrame:
    used_frag_types = frag_types
    predict_intensity_df = predict_intensity_df[
        used_frag_types
    ]

    psm_df, ms2_metrics_df = calc_ms2_similarity(
        psm_df, predict_intensity_df, 
        matched_intensity_df,
        charged_frag_types=used_frag_types,
        metrics=['COS','SA','SPC'],
    )
    psm_df.rename(
        columns={
            'COS':'cos','SA':'sa','SPC':'spc'
        },
        inplace=True
    )

    b_frag_types = [
        _t for _t in used_frag_types 
        if _t.startswith('b')
    ]
    y_frag_types = [
        _t for _t in used_frag_types 
        if _t.startswith('y')
    ]

    frag_position_hits = matched_intensity_df[
        used_frag_types
    ].values.any(axis=1)
    frag_ratio_ion = []
    for start_idx, end_idx in psm_df[
        ['frag_start_idx','frag_end_idx']
    ].values:
        frag_ratio_ion.append(
            np.mean(frag_position_hits[start_idx:end_idx])
        )
    psm_df['frag_ratio'] = frag_ratio_ion

    if len(b_frag_types) > 0:
        psm_df, ms2_metrics_df = calc_ms2_similarity(
            psm_df, predict_intensity_df, 
            matched_intensity_df,
            charged_frag_types=b_frag_types,
            metrics=['COS','SA','SPC'],
        )
        psm_df.rename(
            columns={
                'COS':'cos_bion','SA':'sa_bion','SPC':'spc_bion'
            },
            inplace=True
        )
        frag_position_hits = matched_intensity_df[
            b_frag_types
        ].values.any(axis=1)
        
        frag_ratio_ion = []
        for start_idx, end_idx in psm_df[
            ['frag_start_idx','frag_end_idx']
        ].values:
            frag_ratio_ion.append(
                np.mean(frag_position_hits[start_idx:end_idx])
            )
        psm_df['frag_ratio_bion'] = frag_ratio_ion
    else:
        psm_df[['cos_bion','sa_bion','spc_bion']] = 0
        psm_df[['frag_ratio_bion']] = 0

    if len(y_frag_types) > 0:
        psm_df, ms2_metrics_df = calc_ms2_similarity(
            psm_df, predict_intensity_df, 
            matched_intensity_df,
            charged_frag_types=y_frag_types,
            metrics=['COS','SA','SPC'],
        )
        psm_df.rename(
            columns={
                'COS':'cos_yion','SA':'sa_yion','SPC':'spc_yion'
            },
            inplace=True
        )
        frag_position_hits = matched_intensity_df[
            y_frag_types
        ].values.any(axis=1)
        frag_ratio_ion = []
        for start_idx, end_idx in psm_df[
            ['frag_start_idx','frag_end_idx']
        ].values:
            frag_ratio_ion.append(
                np.mean(frag_position_hits[start_idx:end_idx])
            )
        psm_df['frag_ratio_yion'] = frag_ratio_ion
    else:
        psm_df[['cos_yion','sa_yion','spc_yion']] = 0
        psm_df[['frag_ratio_yion']] = 0

    return psm_df

def match_one_raw_mp(args):
    return match_one_raw(*args)

def get_ms2_features_mp(args):
    return get_ms2_features(*args)


In [3]:
#export

class ScoreFeatureExtractor:
    def __init__(self, 
        model_mgr:ModelManager=None
    ):
        """ ScoreFeatureExtractor: Feature extractor for percolator 
              with a single process.

        Args:
            model_mgr (ModelManager, optional): The external ModelManager.
              If it is None, the models will be init by 
              `ModelManager.load_installed_models()`.
              Defaults to None.
        """
        if model_mgr is None:
            self.model_mgr = ModelManager()
            self.model_mgr.load_installed_models()
        else:
            self.model_mgr = model_mgr

        self.require_model_tuning = True

        self.score_feature_list = [
            'cos','sa','spc',
            'cos_bion','sa_bion','spc_bion',
            'cos_yion','sa_yion','spc_yion',
            'frag_ratio','frag_ratio_bion',
            'frag_ratio_yion','rt_delta_abs', 
            'mobility_delta_abs',
        ]

        self.n_raw_to_tune = 3

    def fine_tune_models(self,
        df_groupby_raw,
        ms2_file_dict,
        ms2_file_type,
        frag_types_to_match,
        ms2_ppm, ms2_tol,
    ):
        if df_groupby_raw.ngroups < self.n_raw_to_tune:
            tune_raw_num = df_groupby_raw.ngroups
        else:
            tune_raw_num = self.n_raw_to_tune

        rs = np.random.RandomState(1337)
        
        raw_list = rs.choice(
            list(df_groupby_raw.groups.keys()), 
            tune_raw_num,
            replace=False
        )

        logging.info('Preparing for fine-tuning ...')
        psm_df_list = []
        matched_intensity_df_list = []
        for raw_name, df in df_groupby_raw:
            if (
                raw_name not in raw_list 
                or raw_name not in ms2_file_dict
            ):
                continue
            
            df = calc_fdr_for_df(df, 'score')
            df = df[(df.fdr<0.01)&(df.decoy==0)]
            (
                df, _, inten_df, _
            ) = match_one_raw(
                df, ms2_file_dict[raw_name],
                ms2_file_type,
                frag_types_to_match,
                ms2_ppm, ms2_tol,
            )
            psm_df_list.append(df)
            matched_intensity_df_list.append(inten_df)

        logging.info('Fine-tuning ...')
        self._tune(
            *concat_precursor_fragment_dataframes(
                psm_df_list,
                matched_intensity_df_list
            )
        )

    def _tune(self,
        psm_df, 
        matched_intensity_df
    ):
        if 'ccs' in psm_df.columns:
            self.model_mgr.fine_tune_ccs_model(psm_df)
        self.model_mgr.fine_tune_rt_model(psm_df)

        self.model_mgr.fine_tune_ms2_model(
            psm_df, matched_intensity_df
        )

    def extract_rt_features(self, psm_df):
        if 'rt_norm' in psm_df.columns:
            psm_df = self.model_mgr.predict_rt(
                psm_df
            )
            psm_df[
                'rt_delta'
            ] = (
                psm_df.rt_pred-psm_df.rt_norm
            )

            psm_df[
                'rt_delta_abs'
            ] = psm_df.rt_delta.abs()
        else:
            psm_df['rt_delta'] = 0
            psm_df['rt_delta_abs'] = 0

    def extract_mobility_features(self, psm_df):
        if (
            'mobility' in psm_df.columns
        ):
            psm_df = self.model_mgr.predict_mobility(
                psm_df
            )

            psm_df = self.ccs_model.ccs_to_mobility_pred(
                psm_df
            )
            
            psm_df[
                'mobility_delta'
            ] = (
                psm_df.mobility_pred-psm_df.mobility
            )
            psm_df[
                psm_df.mobility_delta.isna(),'mobility_delta'
            ] = 0
            psm_df['mobility_delta_abs'] = psm_df.mobility_delta.abs()
        else:
            psm_df['mobility_delta'] = 0
            psm_df['mobility_delta_abs'] = 0
    

    def match_ms2(self,
        psm_df: pd.DataFrame,
        ms2_file_dict, #raw_name: ms2_file_path or ms_reader object
        ms2_file_type:str,
        frag_types_to_match:list = get_charged_frag_types(['b','y'], 2),
        ms2_ppm=True, ms2_tol=20,
    ):
        self.match = PepSpecMatch(
            charged_frag_types=frag_types_to_match
        )

        self.match.match_ms2_centroid(
            refine_precursor_df(psm_df),
            ms2_file_dict=ms2_file_dict,
            ms2_file_type=ms2_file_type, 
            ppm=ms2_ppm, tol=ms2_tol,
        )

    def extract_features(self,
        psm_df: pd.DataFrame,
        ms2_file_dict, 
        ms2_file_type:str = 'alphapept', 
        frag_types_to_match:list = get_charged_frag_types(['b','y'], 2),
        ms2_ppm=True, ms2_tol=20,
    )->pd.DataFrame:
        """ Extract features and add columns (self.score_feature_list) into psm_df

        Args:
            psm_df (pd.DataFrame): psm dataframe to extract features
            ms2_file_dict ([type]): MS2 file path dict: {raw_name: ms2_path}
            ms2_file_type (str, optional): MS2 file type, coult be 
              'alphapept', 'mgf', or 'raw'. Defaults to 'alphapept'.
            frag_types_to_match (list, optional): fragment types. 
              Defaults to ['b_z1','b_z2','y_z1','y_z2'].
            ms2_ppm (bool, optional): Matching MS2 mass tolerance unit. 
              Defaults to True.
            ms2_tol (int, optional): Matching mass tolerance. 
              Defaults to 20.

        Returns:
            pd.DataFrame: psm_df with feature columns added
        """
        
        frag_types = []
        for frag_type in frag_types_to_match:
            if frag_type in self.model_mgr.ms2_model.charged_frag_types:
                frag_types.append(frag_type)

        df_groupby_raw = psm_df.groupby('raw_name')

        if self.require_model_tuning:
            logging.info('Fine-tuning models ...')
            self.fine_tune_models(
                df_groupby_raw, 
                ms2_file_dict, ms2_file_type,
                frag_types, ms2_ppm, ms2_tol
            )

        logging.info('Extracting alphadeep features ...')
        result_psm_list = []
        for raw_name, df in df_groupby_raw:
            if raw_name not in ms2_file_dict:
                continue
            (
                df, frag_mz_df, frag_inten_df, frag_merr_df
            ) = match_one_raw(
                df,
                ms2_file_dict[raw_name],
                ms2_file_type,
                frag_types_to_match,
                ms2_ppm, ms2_tol,
            )

            self.extract_rt_features(df)
            self.extract_mobility_features(df)

            predict_inten_df = self.model_mgr.predict_ms2(
                df, batch_size=512
            )

            result_psm_list.append(
                get_ms2_features(
                    df, frag_types, 
                    predict_inten_df,
                    frag_inten_df,
                    frag_merr_df,
                )
            )
            
        self.psm_df = pd.concat(result_psm_list).reset_index(
            drop=True
        )
        logging.info('Finish extracting features.')
        return self.psm_df
        

In [ ]:
#hide
import torch
from copy import deepcopy

from alphadeep.model.ms2 import pDeepModel
from alphadeep.model.rt import AlphaRTModel
from alphadeep.model.ccs import AlphaCCSModel

def extract_rt_features(psm_df, model:torch.nn.Module):
    if 'rt_norm' in psm_df.columns:
        rt_model = AlphaRTModel()
        rt_model.model = deepcopy(model)
        psm_df = rt_model.predict(
            psm_df
        )
        psm_df[
            'rt_delta'
        ] = (
            psm_df.rt_pred-psm_df.rt_norm
        )

        psm_df[
            'rt_delta_abs'
        ] = psm_df.rt_delta.abs()
    else:
        psm_df['rt_delta'] = 0
        psm_df['rt_delta_abs'] = 0

def extract_mobility_features(psm_df, model:torch.nn.Module):
    if (
        'mobility' in psm_df.columns
    ):
        ccs_model = AlphaCCSModel()
        ccs_model.model = deepcopy(model)
        psm_df = ccs_model.predict(
            psm_df
        )

        psm_df = ccs_model.ccs_to_mobility_pred(
            psm_df
        )
        
        psm_df[
            'mobility_delta'
        ] = (
            psm_df.mobility_pred-psm_df.mobility
        )
        psm_df[
            psm_df.mobility_delta.isna(),'mobility_delta'
        ] = 0
        psm_df['mobility_delta_abs'] = psm_df.mobility_delta.abs()
    else:
        psm_df['mobility_delta'] = 0
        psm_df['mobility_delta_abs'] = 0

def predict_ms2(psm_df, 
    model:torch.nn.Module, 
    frag_types,
):
    pdeep = pDeepModel(frag_types)
    pdeep.model = deepcopy(model)
    return pdeep.predict(psm_df, batch_size=256)

In [4]:
#export
import torch
import torch.multiprocessing as mp

from alphadeep._settings import global_settings
AD_THREAD_NUM = global_settings['thread_num']

class ScoreFeatureExtractorMP(ScoreFeatureExtractor):
    def __init__(self, 
        model_mgr:ModelManager=None
    ):
        """ ScoreFeatureExtractorMP: Feature extractor for percolator 
              with multiprocessing.

        Args:
            model_mgr (ModelManager, optional): The external ModelManager.
              If it is None, the models will be init by 
              `ModelManager.load_installed_models()`.
              Defaults to None.
        """
        super().__init__(model_mgr=model_mgr)
        
        if self.n_raw_to_tune > AD_THREAD_NUM:
            self.n_raw_to_tune = AD_THREAD_NUM
        
        self.mp_pool = mp.Pool(
            processes=AD_THREAD_NUM, 
        )

        self.model_mgr.ms2_model.model.share_memory()
        self.model_mgr.rt_model.model.share_memory()
        self.model_mgr.ccs_model.model.share_memory()


    def fine_tune_models(self,
        df_groupby_raw,
        ms2_file_dict,
        ms2_file_type,
        frag_types_to_match,
        ms2_ppm, ms2_tol,
    ):
        if df_groupby_raw.ngroups < self.n_raw_to_tune:
            tune_raw_num = df_groupby_raw.ngroups
        else:
            tune_raw_num = self.n_raw_to_tune

        rs = np.random.RandomState(1337)
        raw_list = rs.choice(
            list(df_groupby_raw.groups.keys()), 
            tune_raw_num,
            replace=False
        )

        def one_raw_param_generator(df_groupby_raw):
            for raw_name, df in df_groupby_raw:
                if (
                    raw_name not in raw_list 
                    or raw_name not in ms2_file_dict
                ):
                    continue

                df = calc_fdr_for_df(df, 'score')
                df = df[(df.fdr<0.01)&(df.decoy==0)]
                yield (
                    df,
                    ms2_file_dict[raw_name],
                    ms2_file_type,
                    frag_types_to_match,
                    ms2_ppm, ms2_tol,
                )
        
        logging.info('Preparing for fine-tuning ...')  
        psm_df_list = []
        matched_intensity_df_list = []  
        with self.mp_pool as p:
            for df, _, inten_df, _ in p.imap_unordered(
                match_one_raw_mp, 
                one_raw_param_generator(df_groupby_raw)
            ):
                psm_df_list.append(df)
                matched_intensity_df_list.append(inten_df)

        logging.info('Fine-tuning ...')
        self._tune(
            *concat_precursor_fragment_dataframes(
                psm_df_list,
                matched_intensity_df_list
            )
        )

    def extract_features_one_raw(self,
        df_one_raw: pd.DataFrame,
        ms2_file, 
        ms2_file_type,
        frag_types,
        ms2_ppm, ms2_tol,
    ):
        (
            df, frag_mz_df, frag_inten_df, frag_merr_df
        ) = match_one_raw(df_one_raw, 
            ms2_file, ms2_file_type, frag_types,
            ms2_ppm, ms2_tol
        )

        self.extract_rt_features(df)
        self.extract_mobility_features(df)

        predict_inten_df = self.model_mgr.predict_ms2(
            df, batch_size=512
        )

        return get_ms2_features(df, 
            frag_types, 
            predict_inten_df,
            frag_inten_df,
            frag_merr_df,
        )

    def extract_features(self,
        psm_df: pd.DataFrame,
        ms2_file_dict,
        ms2_file_type:str = 'alphapept',
        frag_types_to_match:list = get_charged_frag_types(['b','y'], 2),
        ms2_ppm=True, ms2_tol=20,
    )->pd.DataFrame:

        """ Extract features and add columns (self.score_feature_list) into psm_df

        Args:
            psm_df (pd.DataFrame): psm dataframe to extract features
            ms2_file_dict ([type]): MS2 file path dict: {raw_name: ms2_path}
            ms2_file_type (str, optional): MS2 file type, coult be 
              'alphapept', 'mgf', or 'thermo'. Defaults to 'alphapept'.
            frag_types_to_match (list, optional): fragment types. 
              Defaults to ['b_z1','b_z2','y_z1','y_z2'].
            ms2_ppm (bool, optional): Matching MS2 mass tolerance unit. 
              Defaults to True.
            ms2_tol (int, optional): Matching mass tolerance. 
              Defaults to 20.

        Returns:
            pd.DataFrame: psm_df with feature columns added
        """

        frag_types = []
        for frag_type in frag_types_to_match:
            if frag_type in self.model_mgr.ms2_model.charged_frag_types:
                frag_types.append(frag_type)

        df_groupby_raw = psm_df.groupby('raw_name')

        if self.require_model_tuning:
            logging.info('Fine-tuning models ...')
            self.fine_tune_models(
                df_groupby_raw, 
                ms2_file_dict, ms2_file_type,
                frag_types, ms2_ppm, ms2_tol
            )

        def one_raw_param_generator(df_groupby_raw):
            for raw_name, df in df_groupby_raw:
                if raw_name not in ms2_file_dict:
                    continue
                yield (
                    df,
                    ms2_file_dict[raw_name],
                    ms2_file_type,
                    frag_types_to_match,
                    ms2_ppm, ms2_tol,
                )

        logging.info(
            'Extracting alphadeep features with multiprocessing ...'
        )

        result_psm_list = []
        if torch.cuda.is_available():
            # multiprocessing is only used for ms2 matching
            def prediction_gen(df_groupby_raw):
                with mp.Pool(AD_THREAD_NUM) as _p:
                    for (
                        df, frag_mz_df, frag_inten_df, frag_merr_df
                    ) in _p.imap_unordered(
                        match_one_raw_mp, 
                        one_raw_param_generator(df_groupby_raw)
                    ):

                        self.extract_rt_features(df)
                        self.extract_mobility_features(df)

                        predict_inten_df = self.model_mgr.predict_ms2(
                            df, batch_size=512
                        )

                        yield (
                            df, frag_types, 
                            predict_inten_df,
                            frag_inten_df, frag_merr_df,
                        )

            with self.mp_pool as p:
                for i, df in enumerate(p.imap_unordered(
                    get_ms2_features_mp, 
                    prediction_gen(df_groupby_raw)
                )):
                    result_psm_list.append(df)
                    logging.info(
                        f'Finished extracting features of raw file #{i}'
                    )
        else:
            # use multiprocessing for prediction 
            # only when no GPUs are available
            with self.mp_pool as p:
                for i, _df in enumerate(p.imap_unordered(
                    self.extract_features_one_raw, 
                    one_raw_param_generator(df_groupby_raw)
                )):
                    result_psm_list.append(_df)
                    logging.info(
                        f'Finished extracting features of raw file #{i}'
                    )

        logging.info('Finished feature extraction multiprocessing')

        self.psm_df = pd.concat(result_psm_list).reset_index(
            drop=True
        )
        return self.psm_df

In [6]:
#hide
test=False
if test:
    import os
    from alphabase.io.psm_reader import psm_reader_provider
    hdf_path = os.path.expanduser('~/Workspace/Data/HeLa_500ng/HeLa_500ng.ms_data.hdf')
    reader = psm_reader_provider.get_reader('alphapept', fdr=1, keep_decoy=True)
    reader.import_file(hdf_path)
    select_col = 'spec_idx'
    idx = reader.psm_df.groupby(select_col)['score'].idxmax()
    psm_df = reader.psm_df.loc[idx]
    psm_df.drop_duplicates([select_col,'sequence','mods'],inplace=True)
    psm_df.sort_values(select_col)

    extractor = ScoreFeatureExtractorMP()
    psm_df = extractor.extract_features(psm_df, {'HeLa_500ng':hdf_path})
    display(psm_df)